<a href="https://colab.research.google.com/github/PatrykCiwinski/Fungi_CNN/blob/main/grzyby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import os
import random
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns

In [ ]:
podkategoria = 'grzyby'
tagi = 'jadalne'
strona = 1

In [ ]:
url = f'https://www.ekologia.pl/wiedza/grzyby/tagi,{tagi},s{strona}'

In [ ]:
response = requests.get(url)
content = response.content

In [ ]:
soup = BeautifulSoup(content, 'html.parser')
imgs = soup.select('div.pro_eko img')
urls = [img['src'] for img in imgs]

In [ ]:
urls

['https://zasoby.ekologia.pl/grzyby/a/301/shutterstock-1349274215_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/266/shutterstock-701949721_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/2/640px-boletus-impolitus_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/302/shutterstock-1566904369_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/11/6-borowik-szlachetny-boletus-edulis_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/5/1280px-boletus_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/254/800px-Butyriboletus-persolidus-72425-Novato-California-20091109-retouched_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/67/800px-hand-utri_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/68/czernidlak-kolpakowaty_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/69/shutterstock-1824266690_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/228/shutterstock-1568489815_mini.jpg',
 'https://zasoby.ekologia.pl/grzyby/a/129/px-Chlorophyllum_rachodes_mini.jpg']

In [ ]:
response = requests.get(urls[5])
content = response.content

In [ ]:
img = Image.open(BytesIO(content))

In [ ]:
img.save('grzybnia5.jpg')

In [ ]:
def get_page(tagi,
             strona):
  
  url = f'https://www.ekologia.pl/wiedza/grzyby/tagi,{tagi},s{strona}' 
  response = requests.get(url)
  content = response.content
  return content


def parse_imgs_url(content):
  soup = BeautifulSoup(content, 'html.parser')
  imgs = soup.select('div.pro_eko img')
  return [img['src'] for img in imgs]


def get_image(url, path, tagi):
    response = requests.get(url)
    content = response.content
    img = Image.open(BytesIO(content))
    os.makedirs(f'{path}/{tagi}', exist_ok=True)
    name=url.split('/')[-1].split('.')[0].split('_')[:2]
    name=' '.join(map(str, name))
    img.save(f'{path}/{tagi}/{name}.jpg')


def get_data(tagi,
             start_page,
             end_page,
             path):
  
  for strona in range(start_page, end_page + 1):
    content = get_page(tagi, strona)
    urls = parse_imgs_url(content)

    for i, url in enumerate(urls):
      get_image(url, path, tagi)
      if i % 2 == 0:
        print(f'Downloaded {i} for page {strona} from {tagi}')

In [ ]:
def get_first_page(tagi):
  
  url = f'https://www.ekologia.pl/wiedza/grzyby/tagi,{tagi}' 
  response = requests.get(url)
  content = response.content
  return content


def get_data_first_page(tagi,
             path):
    content = get_first_page(tagi)
    urls = parse_imgs_url(content)
  

In [ ]:
data_path = '/content/drive/MyDrive/grzyby1/data/train'

In [ ]:
get_data('jadalne',1,16,path = data_path)

In [ ]:
get_data('niejadalne',1,16,path = data_path)

Downloaded 0 for page 1 from niejadalne
Downloaded 2 for page 1 from niejadalne
Downloaded 4 for page 1 from niejadalne
Downloaded 6 for page 1 from niejadalne
Downloaded 8 for page 1 from niejadalne
Downloaded 10 for page 1 from niejadalne
Downloaded 0 for page 2 from niejadalne
Downloaded 2 for page 2 from niejadalne
Downloaded 4 for page 2 from niejadalne
Downloaded 6 for page 2 from niejadalne
Downloaded 8 for page 2 from niejadalne
Downloaded 10 for page 2 from niejadalne
Downloaded 0 for page 3 from niejadalne
Downloaded 2 for page 3 from niejadalne
Downloaded 4 for page 3 from niejadalne
Downloaded 6 for page 3 from niejadalne
Downloaded 8 for page 3 from niejadalne
Downloaded 10 for page 3 from niejadalne
Downloaded 0 for page 4 from niejadalne
Downloaded 2 for page 4 from niejadalne
Downloaded 4 for page 4 from niejadalne
Downloaded 6 for page 4 from niejadalne
Downloaded 8 for page 4 from niejadalne
Downloaded 10 for page 4 from niejadalne
Downloaded 0 for page 5 from niejada

In [ ]:
get_data('trujace',1,16,path = data_path)

Downloaded 0 for page 1 from trujace
Downloaded 2 for page 1 from trujace
Downloaded 4 for page 1 from trujace
Downloaded 6 for page 1 from trujace
Downloaded 8 for page 1 from trujace
Downloaded 10 for page 1 from trujace
Downloaded 0 for page 2 from trujace
Downloaded 2 for page 2 from trujace
Downloaded 4 for page 2 from trujace
Downloaded 6 for page 2 from trujace
Downloaded 8 for page 2 from trujace
Downloaded 10 for page 2 from trujace
Downloaded 0 for page 3 from trujace
Downloaded 2 for page 3 from trujace
Downloaded 4 for page 3 from trujace
Downloaded 6 for page 3 from trujace
Downloaded 8 for page 3 from trujace
Downloaded 10 for page 3 from trujace
Downloaded 0 for page 4 from trujace
Downloaded 2 for page 4 from trujace
Downloaded 4 for page 4 from trujace
Downloaded 6 for page 4 from trujace
Downloaded 8 for page 4 from trujace
Downloaded 10 for page 4 from trujace
Downloaded 0 for page 5 from trujace


In [ ]:
get_data_first_page('jadalne',path = data_path)

In [ ]:
get_data_first_page('niejadalne',path = data_path)

In [ ]:
get_data_first_page('trujace',path = data_path)

In [ ]:
data_path = '/content/drive/MyDrive/grzyby1/data'
os.makedirs(data_path + '/test', exist_ok=True)
test_size = 0.2

for subcategory in ['jadalne', 'niejadalne', 'trujace']:
  os.makedirs(data_path + '/test/' + subcategory, exist_ok=True)
  file_list = os.listdir(data_path + '/train/' + subcategory)
  number_of_samples = int(len(file_list) * test_size)
  for file_name in random.sample(file_list, number_of_samples):
    os.rename(data_path + '/train/' + subcategory + '/' + file_name,
              data_path + '/test/' + subcategory + '/' + file_name)

In [ ]:
len(os.listdir(data_path + '/train/jadalne'))

136

In [ ]:
from tensorflow import keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import keras.utils as image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
dim1 = []
dim2 = []
for name in os.listdir('/content/drive/MyDrive/grzyby1/data/train/jadalne'):
    
    img = imread('/content/drive/MyDrive/grzyby1/data/train/jadalne'+'/'+name)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
np.mean(dim1)

115.0

In [ ]:
np.mean(dim2)

170.03676470588235

In [ ]:
image_shape = (115,170,3)

In [ ]:
batch_size=24

In [ ]:
image_gen.flow_from_directory('/content/drive/MyDrive/grzyby1/data/train')

Found 315 images belonging to 3 classes.


In [ ]:
image_gen.flow_from_directory('/content/drive/MyDrive/grzyby1/data/test')

Found 137 images belonging to 3 classes.


In [ ]:
train_image_gen = image_gen.flow_from_directory('/content/drive/MyDrive/grzyby1/data/train',
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 315 images belonging to 3 classes.


In [ ]:
test_image_gen = image_gen.flow_from_directory('/content/drive/MyDrive/grzyby1/data/test',
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 137 images belonging to 3 classes.


In [ ]:
train_image_gen.class_indices

{'jadalne': 0, 'niejadalne': 1, 'trujace': 2}

In [ ]:
model = Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))


model.add(layers.Flatten())


model.add(layers.Dense(128))
model.add(layers.Activation('relu'))

model.add(layers.Dropout(0.5))


model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
history = model.fit(train_image_gen,epochs=200,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

In [ ]:
my_image = image.load_img('/content/drive/MyDrive/grzyby1/data/train/trujace/800px-paxillus-involutus-g2 mini.jpg',target_size=image_shape)

In [ ]:
my_image = image.img_to_array(my_image)

In [ ]:
my_image = np.expand_dims(my_image, axis=0)

In [ ]:
my_image.shape

In [ ]:
predict=model.predict(my_image)
predictions=np.argmax(predict,axis=1)

In [ ]:
predictions

array([1])